# 第十四次课后练习

**负责助教：胡天成**

<span style="color:red; font-weight:bold;">请将作业文件命名为 第十四次课后练习+姓名+学号.ipynb, 例如 第十四次课后练习+张三+1000000000.ipynb</span>

<span style="color:red; font-weight:bold;">在作业过程中觉得有心得或者自己拓展学习到有价值内容的，可以在文件名最后加一个#号。例如第十四次课后练习+张三+1000000000+#.ipynb</span>

# 第零部分 代码理解

请认真阅读代码，理解代码的功能，先写出预想的结果。运行并检验结果是否如预期。如果不如预期，请分析理解其中的原因

## **0.1** 高斯拉普拉斯算子

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np

from torch.autograd import Variable
from math import exp
from PIL import Image
from torchvision.utils import save_image, make_grid

#将最后的矩阵中的元素归一化到0~1之间
def minmaxscaler(data):
    min = torch.min(data)
    max = torch.max(data)    
    return (data - min)/(max-min)

#LoG变换
def LoG(img,window,window_size,mode="RGB"):
    img1_array = np.array(img,dtype=np.float32)#Image -> array
    img1_tensor = torch.from_numpy(img1_array)# array -> tensor
    # 处理不同通道数的数据
    if mode == 'L':
        img1_tensor = img1_tensor.unsqueeze(0).unsqueeze(0)#h,w -> n,c,h,w 
    else:#RGB or RGBA
        img1_tensor = img1_tensor.permute(2,0,1)# h,w,c -> c,h,w 
        img1_tensor = img1_tensor.unsqueeze(0)#c,h,w -> n,c,h,w 
    channel = img1_tensor.size()[1]
    window = Variable(window.expand(channel, 1, window_size, window_size).contiguous())
    output = F.conv2d(img1_tensor, window, padding = window_size//2, groups = channel)
    output = minmaxscaler(output)# 归一化到0~1之间
    if (channel==4):
        save_image(output, "0-1-output.png", normalize=False)
    else:
        save_image(output, "0-1-output.png", normalize=False)
    return output

#近似卷积核
window  = torch.Tensor([[[0,1,1,2,2,2,1,1,0],
                        [1,2,4,5,5,5,4,2,1],
                        [1,4,5,3,0,3,5,4,1],
                        [2,5,3,-12,-24,-12,3,5,2],
                        [2,5,0,-24,-40,-24,0,5,2],
                        [2,5,3,-12,-24,-12,3,5,2],
                        [1,4,5,3,0,3,4,4,1],
                        [1,2,4,5,5,5,4,2,1],
                        [0,1,1,2,2,2,1,1,0]]])
window_size = 9
img = Image.open("./0-1-input.png")
img = img.convert('L') 
LoG(img,window,window_size,img.mode)

![jupyter](./0-1-input.png)


![jupyter](./0-1-output.png)

这部分代码对输入图片做 Laplacian‑of‑Gaussian 卷积滤波, 并将结果归一化到 [0,1] 后保存为图片, 实现 LoG 边缘检测增强.

## **0.2** 高斯模糊

In [ ]:
import cv2

# 1. 读取图像（支持彩色/灰度图）
image = cv2.imread("0-1-input.png")  # 替换为你的图像路径

# 2. 应用高斯模糊
blurred = cv2.GaussianBlur(
    src=image, 
    ksize=(15, 15),   # 高斯核尺寸（必须为奇数）
    sigmaX=0          # X方向标准差（0表示自动计算）
)

cv2.imwrite("0-2-output.png", blurred)  # 保存模糊图像

![jupyter](./0-2-output.png)

这部分代码实现了高斯模糊.

## **0.3** 仿射变换

### 图像平移

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('0-3-input.png')
H = np.float32([[1,0,100],[0,1,50]])
rows,cols = img.shape[:2]
res = cv2.warpAffine(img,H,(rows,cols)) #需要图像、变换矩阵、变换后的大小
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(res)

### 图像的扩大与缩小

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('0-3-input.png')

res1 = cv2.resize(img,None,fx=2,fy=2,interpolation=cv2.INTER_CUBIC)
#直接规定缩放大小，这个时候就不需要缩放因子
height,width = img.shape[:2]
res2 = cv2.resize(img,(4*width,4*height),interpolation=cv2.INTER_CUBIC)
plt.subplot(131)
plt.imshow(img)
plt.subplot(132)
plt.imshow(res1)
plt.subplot(133)
plt.imshow(res2)


这部分代码将图像在水平方向右移 100 像素, 垂直方向下移 50 像素, 并展示结果.

### 图像的旋转

In [ ]:
import cv2
import matplotlib.pyplot as plt

img = cv2.imread('0-3-input.png')
rows,cols = img.shape[:2]
#第一个参数旋转中心，第二个参数旋转角度，第三个参数：缩放比例
M = cv2.getRotationMatrix2D((cols/2,rows/2),45,1)
#第三个参数：变换后的图像大小
res = cv2.warpAffine(img,M,(rows,cols))

plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(res)

这部分代码对图像进行了 45 度旋转.

### 图像的仿射

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('0-3-input.png')
rows,cols = img.shape[:2]
pts1 = np.float32([[50,50],[200,50],[50,200]])
pts2 = np.float32([[10,100],[200,50],[100,250]])
M = cv2.getAffineTransform(pts1,pts2)
#第三个参数：变换后的图像大小
res = cv2.warpAffine(img,M,(rows,cols))
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(res)

这部分代码指定了三个点并进行了仿射变换.

### 图像的透射

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread('0-3-input.png')
rows,cols = img.shape[:2]
pts1 = np.float32([[56,65],[238,52],[28,237],[239,240]])
pts2 = np.float32([[0,0],[200,0],[0,200],[200,200]])
M = cv2.getPerspectiveTransform(pts1,pts2)
res = cv2.warpPerspective(img,M,(200,200))
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(res)

这部分代码把图像中的一个四边形区域, 通过透视变换"校正"为规则矩形.

## **0.4** DCT变换

In [ ]:
import cv2
import numpy as np

img = cv2.imread("0-3-input.png")
img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
y_channel = img_yuv[:,:,0].astype(np.float32)

block_size = 8
h, w = y_channel.shape

# 增强填充量以保持分块对齐
pad_h = (block_size - h % block_size) % block_size
pad_w = (block_size - w % block_size) % block_size
padded = cv2.copyMakeBorder(y_channel, 0, pad_h, 0, pad_w, cv2.BORDER_CONSTANT)

# 强化版量化表（右下角高频区域量化步长显著增大）
quantization_table = np.array([
    [16,  20,  25,  32,  45,  70,  90, 120],
    [20,  24,  30,  40,  55,  90, 110, 140],
    [25,  30,  40,  50,  65, 100, 130, 160],
    [32,  40,  50,  60,  80, 120, 150, 180],
    [45,  55,  65,  80, 100, 140, 170, 200],
    [70,  90, 100, 120, 140, 180, 210, 240],
    [90, 110, 130, 150, 170, 210, 240, 270],
    [120,140, 160, 180, 200, 240, 270, 300]
], dtype=np.float32)

quality = 0.05  # 降低质量因子
quant_table = quantization_table * (3 - quality*3)  # 增强量化强度系数

compressed_blocks = []
for i in range(0, padded.shape[0], block_size):
    for j in range(0, padded.shape[1], block_size):
        block = padded[i:i+block_size, j:j+block_size]
        
        # 增强中心化处理
        centered = block - 128
        
        # DCT变换
        dct_block = cv2.dct(centered)
        
        # 增强量化（增加高频截断）
        quantized = np.floor(dct_block / quant_table + 0.5)  # 改用floor取整
        quantized[np.abs(quantized) < 0.7] = 0  # 新增小系数截断
        
        # 反量化重建
        dequantized = quantized * quant_table
        idct_block = cv2.idct(dequantized) + 128
        
        compressed_blocks.append(idct_block)

# 图像重建（相同部分省略）
reconstructed = np.zeros_like(padded)
idx = 0
for i in range(0, padded.shape[0], block_size):
    for j in range(0, padded.shape[1], block_size):
        reconstructed[i:i+block_size, j:j+block_size] = compressed_blocks[idx]
        idx += 1

reconstructed_y = reconstructed[:h, :w].clip(0, 255).astype(np.uint8)
compressed_yuv = img_yuv.copy()
compressed_yuv[:,:,0] = reconstructed_y
compressed_bgr = cv2.cvtColor(compressed_yuv, cv2.COLOR_YUV2BGR)

cv2.imwrite("0-3-output-high-compression.png", compressed_bgr)

mse = np.mean((y_channel - reconstructed[:h, :w])**2)
psnr = 10 * np.log10(255**2 / mse)
print(f"PSNR: {psnr:.2f} dB")

![jupyter](./0-3-input.png) ![jupyter](./0-3-output-high-compression.png)


这部分代码展示了数字图像压缩, 进行色彩空间转换和空间预处理, 并DCT变换与量化, 最后重建图像.

## 0.5 短视频生成

In [14]:
import os
import numpy as np
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, Video, clear_output

# 新版 MoviePy 2.0+ 导入方式
from moviepy import *


# 配置参数
input_files = ["horse1.jpg", "horse2.jpg", "horse3.jpg", "horse4.jpg"]
media_dir = "pic/"
output_path = os.path.join(media_dir, "output_video.mp4")
audio_path = os.path.join(media_dir, "胡萝卜2m.wav")
target_resolution = (800, 600)

# 创建交互控件
duration_controls = [
    widgets.FloatSlider(
        value=1.0,
        min=0.5,
        max=5.0,
        step=0.1,
        description=f'{file} Duration:',
        style={'description_width': 'initial'}
    ) for file in input_files
]

ui = widgets.VBox([
    widgets.HBox([
        widgets.FloatSlider(value=2.0, min=0.5, max=5.0, step=0.1,
                          description='Global Duration:'),
        widgets.Checkbox(value=False, description='Keep Aspect Ratio')
    ]),
    *duration_controls,
    widgets.Button(description="Render Video", button_style='success')
])

display(ui)

def process_media(b):
    clear_output(wait=True)
    display(ui)
    
    try:
        # 参数获取
        global_dur = ui.children[0].children[0].value
        keep_ratio = ui.children[0].children[1].value
        durations = [ctrl.value for ctrl in duration_controls]

        # 1. 文件验证
        if not all(os.path.exists(os.path.join(media_dir, f)) for f in input_files):
            raise FileNotFoundError("Missing image files")
        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"Audio file missing: {audio_path}")

        # 2. 图像处理
        processed = []
        for file in input_files:
            img_path = os.path.join(media_dir, file)
            with Image.open(img_path) as img:
                img = img.convert("RGB")
                
                if keep_ratio:
                    img.thumbnail(target_resolution, Image.Resampling.LANCZOS)
                    canvas = Image.new("RGB", target_resolution, (0, 0, 0))
                    offset = (
                        (target_resolution[0] - img.width) // 2,
                        (target_resolution[1] - img.height) // 2
                    )
                    canvas.paste(img, offset)
                else:
                    canvas = img.resize(target_resolution, Image.Resampling.LANCZOS)
                    
                processed.append(np.array(canvas))

        # 3. 视频合成
        clips = [
            ImageClip(img_array, duration=dur)  # 直接指定时长
            for img_array, dur in zip(processed, durations)
        ]
        
        video = concatenate_videoclips(clips, method="chain")
        
        # 4. 音频处理
        audio = AudioFileClip(audio_path)
        final_duration = min(video.duration, audio.duration)
        video = video.subclipped(0, final_duration)
        audio = audio.subclipped(0, final_duration)
        
        # 5. 输出视频
        video.write_videofile(
            output_path,
            fps=24,
            codec="libx264",
            audio_codec="aac",
            threads=4,
            logger=None  # 禁用日志输出
        )
        
        # 显示结果
        print(f"✅ Video rendered ({final_duration:.1f}s)")
        display(Video(output_path, embed=True))
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

ui.children[-1].on_click(process_media)

✅ Video rendered (4.0s)


这部分代码提供了一个可视化界面, 把多张图片做成一段带背景音乐的幻灯片视频, 并且可以随时调整每张图的时长和是否保持纵横比.

# 第一部分 代码填空

基于HOG特征和SVM实现cifar10分类

1. 下载cifar10数据集并解压

In [ ]:
import os
import pickle
import numpy as np
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
import random

# 手动下载并解压原始二进制文件
def download_cifar10():
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    save_path = "./cifar-10-python.tar.gz"
    
    # 下载压缩包（约163MB）
    if not os.path.exists(save_path):
        print("下载数据集中...")
        urlretrieve(url, save_path)
    
    # 解压文件
    if not os.path.exists("./cifar-10-batches-py"):
        import tarfile
        with tarfile.open(save_path, 'r:gz') as tar:
            tar.extractall()

# 数据加载函数
def load_batch(file_path):
    with open(file_path, 'rb') as f:
        batch = pickle.load(f, encoding='bytes')
    return batch

# 执行下载
download_cifar10()

# 加载训练数据并显示前五张图片
# 加载第一个训练批次
batch = load_batch('./cifar-10-batches-py/data_batch_1')
images = batch[b'data'][:5]  # 取前5个样本
labels = batch[b'labels'][:5]

# 转换图像格式 (NCHW -> NHWC)
images = images.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

# 显示图像
plt.figure(figsize=(10, 3))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(images[i])
    plt.title(f'Label: {labels[i]}')
    plt.axis('off')
plt.show()

2. 提取HOG特征,注意提取特征之前需要吧rgb图像先转换成灰度图像，输出给的是mac air运行的一个参考特征提取时间

In [ ]:
from skimage.feature import hog
import numpy as np
import joblib
import os
import time
import pickle as pk


def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def getData(filePath):
    TrainData = []
    for childDir in os.listdir(filePath):
        if 'data_batch' in childDir:
            f = os.path.join(filePath, childDir)
            data = unpickle(f)
            train = np.reshape(data[b'data'], (10000, 3, 32 * 32))
            labels = np.reshape(data[b'labels'], (10000, 1))
            fileNames = np.reshape(data[b'filenames'], (10000, 1))
            datalebels = zip(train, labels, fileNames)
            TrainData.extend(datalebels)
        if childDir == "test_batch":
            f = os.path.join(filePath, childDir)
            data = unpickle(f)
            test = np.reshape(data[b'data'], (10000, 3, 32 * 32))
            labels = np.reshape(data[b'labels'], (10000, 1))
            fileNames = np.reshape(data[b'filenames'], (10000, 1))
            TestData = zip(test, labels, fileNames)
    return TrainData, TestData


def getFeat(TrainData, TestData):
    # 定义hot特征提取的必要参数
    orientations = 9
    pixels_per_cell = (8, 8)
    cells_per_block = (2, 2)

    # 创建保存目录
    os.makedirs('./test', exist_ok=True)
    os.makedirs('./train', exist_ok=True)

    for data in TestData:
        # 提取hog特征
        img_data = data[0]  # 形状为 (3, 1024)
        label = data[1][0]
        filename = data[2][0].decode('utf-8').split('.')[0]
        
        # 转换为标准图像格式 (32x32x3)
        img = img_data.reshape(3, 32, 32).transpose(1, 2, 0)
        
        # 转换为灰度图
        gray_img = rgb2gray(img)
        
        # 提取HOG特征
        fd = hog(gray_img, orientations=orientations,
                 pixels_per_cell=pixels_per_cell,
                 cells_per_block=cells_per_block,
                 channel_axis=None)
        
        # 保存特征和标签
        joblib.dump((fd, label), f'./test/{filename}.feat')
    print("Test features are extracted and saved.")
    
    for data in TrainData:
        # 处理训练集 (同上)
        img_data = data[0]
        label = data[1][0]
        filename = data[2][0].decode('utf-8').split('.')[0]
        
        img = img_data.reshape(3, 32, 32).transpose(1, 2, 0)
        gray_img = rgb2gray(img)
        
        fd = hog(gray_img, orientations=orientations,
                 pixels_per_cell=pixels_per_cell,
                 cells_per_block=cells_per_block,
                 channel_axis=None)
        
        joblib.dump((fd, label), f'./train/{filename}.feat')
    print("Train features are extracted and saved.")

def rgb2gray(im):
    gray = im[:, :, 0]*0.2989+im[:, :, 1]*0.5870+im[:, :, 2]*0.1140
    return gray

if __name__ == '__main__':
    t0 = time.time()
    filePath = r'./cifar-10-batches-py'
    TrainData, TestData = getData(filePath)
    getFeat(TrainData, TestData)
    t1 = time.time()
    print("Features are extracted and saved.")
    print('The cast of time is:%f'%(t1-t0))

3. 训练SVM进行图像分类，正确率50%左右，处理时间大概一分钟，思考怎么优化，或者尝试之前上课讲过的别的分类方法

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np
import glob
import os
import time

train_feat_path = './train/'
test_feat_path = './test/'
if __name__ == "__main__":
    t0 = time.time()
    fds = []
    labels = []
    num = 0
    total = 0
    for feat_path in glob.glob(os.path.join(train_feat_path, '*.feat')):
        data = joblib.load(feat_path)
        fds.append(data[:-1])
        labels.append(data[-1])

    # 训练一个SVM分类器，并用测试集进行测试
    # 加载训练特征
    fds, labels = [], []
    for feat_path in glob.glob(os.path.join(train_feat_path, '*.feat')):
        data = joblib.load(feat_path)
        fds.append(data[0])
        labels.append(data[1])

    X_train = np.array(fds)
    y_train = np.array(labels)

    # 标准化特征
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)

    # 训练SVM
    clf = LinearSVC(C=0.1, max_iter=10000)  # 调参后结果
    clf.fit(X_train, y_train)

    # 加载测试特征
    X_test, y_test = [], []
    for feat_path in glob.glob(os.path.join(test_feat_path, '*.feat')):
        data = joblib.load(feat_path)
        X_test.append(data[0])
        y_test.append(data[1])

    X_test = scaler.transform(np.array(X_test))
    y_test = np.array(y_test)

    # 评估准确率
    y_pred = clf.predict(X_test)
    rate = accuracy_score(y_test, y_pred)
    print(f'The classification accuracy is {rate:.4f}')
    
    t1 = time.time()
    print('The classification accuracy is %f'%rate)
    print('The cast of time is :%f'%(t1-t0))